In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

import cv2
from cv2 import dilate
from cv2 import adaptiveThreshold

from os import listdir
from os.path import isfile, join

from skimage.morphology import *
from skimage import io

In [2]:
import pickle
    
with open ('outfile', 'rb') as fp:
    data = pickle.load(fp)

In [3]:
def ImageToFeatures(img):
    data = []
    for i, row in enumerate(img):
        for j, val in enumerate(row):
            if val:
                data.append([j, i])
    return data

In [155]:
stat=[]
for notch in data:    
    fts = np.array(ImageToFeatures(notch))
    area = len(fts)
    left = min(fts[:,0])
    top = max(fts[:,1])
    right = max(fts[:,0])
    bottom = min(fts[:,1])
    width = right - left
    height = top - bottom
    stat.append([left, top, width, height, area])

In [156]:
labels = [
    #0
    2, 0, 1, 2, 0, #4
    #1
    2, 1, 2, 5, 0, #9
    #2
    2, 5, 5, 0, 1, 0, #15
    #3
    5, 5, 5, 1, 0, #20
    #4
    5, 5, 5, 1, #24
    #5
    5, 5, 5, 1, #28
    #6    
    5, 5, 5, 1, #32
    #7
    5, 5, 5, 1, #36
    #8
    5, 5, 5, 1, #40
    #9
    5, 5, 5, 1, #44
    #10
    5, 5, 5, 1, #48
    #11
    5, 5, 5, 1, #52
    #12
    5, 5, 5, 1, #56
    #13
    5, 5, 5, 0, 1, #61
    #14
    5, 5, 5, 0, 1, #66
    #15
    2, 5, 5, 0, 1, #71
    #16
    2, 5, 1, 4, 2, 5, 1, #78
    #17    
    0, 2, 1, 5, 0, 0, 4, 2, 0, #87
    #18
    4, 0, 1, 0, 2, 0, 0, 5, 0, 0, 5, 1, #99
    #19
    4, 0, 1, 1, 5, 2, 1, #106
    #20
    4, 1, 1, 4, 0, 1, 0, #113
    #21
    4, 1, 0, 4, 1, 4, #119
    #22
    4, 1, 0, 1, 4, 1, 1, 4, 0, #128
    #23
    4, 0, 0, 1, 4, 5, 0, 4, 1, #137
    #24
    4, 1, 1, 1, 4, 0, 0, 4, 1, #146
    #25
    2, 0, 0, 1, 5, 4, 1, 1, 4, 1, #156
    #26
    2, 1, 1, 0, 2, 1, 0, 4,  #164
    #27
    0, 0, 1, 0, 1, 2, 0, 2, 1, #173
    #28
    1, 1, 2, 5, 1, #178
    #29    
    0, 1, 5, 2, 5, 1, 1, #184
    #30
    1, 5, 5, 5, 1, 1, #190
    #31
    0, 0, 5, 5, 5, 1, 1, #197
    #32
    1, 5, 5, 5, 1, 0, #203
    #33
    0, 5, 5, 5, 0, 1, 0, #210
    #34
    5, 5, 5, 0, 0, 1, 1, #217
    #35
    5, 5, 5, 0, 0, #222
    #36
    5, 5, 5, 1, 0, #227
    #37
    5, 5, 5, 0, 0, #232
    #38
    5, 5, 5, 0, 0, #237
    #39
    5, 5, 5, 0, 0, 0, #243
    #40
    5, 5, 5, 0, 0, 0, #249
    #41
    5, 5, 2, 0, 0, #254
    #42
    1, 5, 2, 2, 0, #259 !
    #43
    1, 2, 2, 0, 1, 0, #266 !
    #44
    1, 2, 2, 0, 1, 0, #272
    #45
    2, 4, 0, 1, 4, 0, #278
    #46
    4, 0, 4, 1, 0, 4, 1, #285
    #47    
    4, 0, 0, 4, 5, 1, 0, 1, #295
    #48
    4, 0, 4, 0, 0, 0, 1, 
    #49
    4, 0, 0, 4, 0, 0, 1,
    #50
    4, 0, 0, 4, 1, 0, 1, 
    #51
    4, 0, 4, 1, 0, 1, 
    #52
    4, 0, 0, 4, 1, 1, 0, #327
    #53
    4, 0, 2, 5, 1, 0, 1, 1    
]

0 - мусор<br>
1 - засвет<br>
2 - крест<br>
3 - круг<br>
4 - горизонтальная засечка<br>
5 - вертикальная засечка<br>

In [157]:
labels = [0 if l in (0, 1, 2, 3) else 1 for l in labels]

In [158]:
stat=np.array(stat)
stat = stat[:,2:5]

# Random forest classifier

In [221]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [222]:
clf = RandomForestClassifier()
clf.fit(stat, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [223]:
sum(clf.predict(stat) == labels) / len(labels)

1.0

In [224]:
X_train, X_test, y_train, y_test = train_test_split(stat, labels)
clf.fit(X_train, y_train)
sum(clf.predict(X_test) == y_test)/len(y_test)

0.9761904761904762

In [225]:
from sklearn.externals import joblib
joblib.dump(clf, 'model.pkl')

['model.pkl']